# Финальный ноутбук



In [6]:
# ============================================================================
# ЯЧЕЙКА 1: Установка зависимостей и импорт библиотек
# ============================================================================
print("=" * 80)
print("СИСТЕМА NER ДЛЯ ЮРИДИЧЕСКИХ ДОКУМЕНТОВ")
print("=" * 80)

# Установка необходимых библиотек (если не установлены)
try:
    import torch
except ImportError:
    !pip install -q torch numpy

import torch
import torch.nn as nn
import json
import re
import numpy as np
from collections import defaultdict
import os

BASE_DIR = os.path.dirname(os.path.abspath("__file__")) \
    if "__file__" in globals() else os.getcwd()

os.chdir(BASE_DIR)

from datetime import datetime
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")
print()

# ============================================================================
# ЯЧЕЙКА 2: КОНСТАНТЫ И НАСТРОЙКИ (вынесены в конфигурацию)
# ============================================================================
class Config:
    """Конфигурация системы NER"""
    # Параметры обработки
    MAX_LEN = 256  # Размер окна для обработки
    STRIDE = 128   # Шаг скользящего окна

    # Цвета для классов сущностей
    LABEL_COLORS = {
        "s1": "#1976d2",  # синий - обязательства и права
        "s2": "#388e3c",  # зелёный - деньги, суммы и проценты
        "s3": "#fbc02d",  # жёлтый - даты и периоды
        "s4": "#d32f2f",  # красный - адреса
        "O": "#ffffff"    # белый - не сущность
    }

    # Пути к файлам по умолчанию (относительные, в той же папке)
    DEFAULT_PATHS = {
        "model": "bilstm_ner_best.pt",           # модель
        "char_vocab": "char_vocab.json",         # словарь символов
        "label_vocab": "label_vocab.json",       # словарь меток
        "sample_text": "sample_contract.txt"     # пример текста для демо
    }

    # Описание классов
    CLASS_DESCRIPTIONS = {
        "O": "Не сущность",
        "s1": "Обязательства, права, субъекты",
        "s2": "Деньги, суммы, проценты",
        "s3": "Даты, сроки, периоды",
        "s4": "Адреса, места, локации"
    }

    # Параметры модели
    EMBEDDING_DIM = 128
    HIDDEN_DIM = 128

config = Config()
print("✅ Конфигурация загружена")
print()

# ============================================================================
# ЯЧЕЙКА 3: АРХИТЕКТУРА МОДЕЛИ BiLSTM (без изменений)
# ============================================================================
class BiLSTMNER(nn.Module):
    """BiLSTM модель для распознавания именованных сущностей (NER)"""

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_labels, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=pad_idx
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_dim * 2, num_labels)

    def forward(self, x):
        emb = self.embedding(x)
        out, _ = self.lstm(emb)
        logits = self.fc(out)
        return logits

print("✅ Архитектура модели определена")
print()

# ============================================================================
# ЯЧЕЙКА 4: ФУНКЦИЯ ЗАГРУЗКИ МОДЕЛИ И СЛОВАРЕЙ (исправлена)
# ============================================================================
def load_model_and_vocabs(model_path=None, char_vocab_path=None, label_vocab_path=None):
    """
    Загрузка обученной модели и словарей

    Параметры:
    - model_path: путь к файлу модели (.pt) или None для использования по умолчанию
    - char_vocab_path: путь к словарю символов (.json)
    - label_vocab_path: путь к словарю меток (.json)

    Возвращает:
    - model: загруженная модель
    - char2idx: словарь символ->индекс
    - idx2label: словарь индекс->метка
    - device: устройство (cpu/cuda)
    """
    # Используем пути по умолчанию, если не указаны
    if model_path is None:
        model_path = config.DEFAULT_PATHS["model/bilstm_ner_best.pt"]
    if char_vocab_path is None:
        char_vocab_path = config.DEFAULT_PATHS["vocab/char_vocab.json"]
    if label_vocab_path is None:
        label_vocab_path = config.DEFAULT_PATHS["vocab/label_vocab.json"]

    print(f"🔄 Загрузка модели из: {model_path}")
    print(f"🔄 Загрузка словаря символов из: {char_vocab_path}")
    print(f"🔄 Загрузка словаря меток из: {label_vocab_path}")

    try:
        # Проверяем существование файлов
        for path, name in [(model_path, "модель"),
                          (char_vocab_path, "словарь символов"),
                          (label_vocab_path, "словарь меток")]:
            if not os.path.exists(path):
                raise FileNotFoundError(f"Файл {name} не найден: {path}")

        # Загрузка словарей
        with open(char_vocab_path, 'r', encoding='utf-8') as f:
            char2idx = json.load(f)

        with open(label_vocab_path, 'r', encoding='utf-8') as f:
            label2idx = json.load(f)

        # Создание обратного словаря меток
        idx2label = {int(v): k for k, v in label2idx.items()}

        # Параметры модели
        vocab_size = len(char2idx)
        num_labels = len(label2idx)
        pad_idx = char2idx.get('<PAD>', 0)

        print(f"📊 Размер словаря символов: {vocab_size}")
        print(f"📊 Количество классов: {num_labels}")

        # Создание и загрузка модели
        model = BiLSTMNER(
            vocab_size=vocab_size,
            embedding_dim=config.EMBEDDING_DIM,
            hidden_dim=config.HIDDEN_DIM,
            num_labels=num_labels,
            pad_idx=pad_idx
        )

        # Загрузка весов
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()

        print(f"✅ Модель загружена на устройство: {device}")
        print(f"✅ Словари загружены")

        return model, char2idx, idx2label, device

    except FileNotFoundError as e:
        print(f"❌ Ошибка: файл не найден - {e}")
        print("💡 Убедитесь, что следующие файлы находятся в той же папке, что и ноутбук:")
        for key, filename in config.DEFAULT_PATHS.items():
            if key != "sample_text":  # sample_text опциональный
                print(f"   - {filename}")
        raise
    except Exception as e:
        print(f"❌ Ошибка при загрузке: {e}")
        raise

print("✅ Функция загрузки модели определена")
print()

# ============================================================================
# ЯЧЕЙКА 5: ФУНКЦИЯ ПРЕДОБРАБОТКИ ТЕКСТА (чистая функция)
# ============================================================================
def preprocess_text(text, lower=True):
    """Предобработка текста для модели"""
    if not text or not isinstance(text, str):
        return ""

    # Удаление лишних пробелов
    text = ' '.join(text.split())

    # Приведение к нижнему регистру (опционально)
    if lower:
        text = text.lower()

    # Замена переносов строк на пробелы
    text = text.replace('\n', ' ').replace('\r', ' ')

    # Удаление множественных пробелов
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

print("✅ Функция предобработки текста определена")
print()

# ============================================================================
# ЯЧЕЙКА 6: ФУНКЦИЯ ТОКЕНИЗАЦИИ (чистая функция)
# ============================================================================
def text_to_indices(text, char2idx, max_len=None):
    """
    Преобразование текста в индексы для модели

    Параметры:
    - text: предобработанный текст
    - char2idx: словарь символ->индекс
    - max_len: максимальная длина последовательности (по умолчанию из config)

    Возвращает:
    - indices: список индексов
    - chars: список символов
    """
    if max_len is None:
        max_len = config.MAX_LEN

    # Разбиваем текст на символы
    chars = list(text)

    # Преобразуем символы в индексы
    unk_idx = char2idx.get('<UNK>', 0)
    pad_idx = char2idx.get('<PAD>', 0)

    indices = [char2idx.get(c, unk_idx) for c in chars]

    # Обрезаем до max_len
    indices = indices[:max_len]
    chars = chars[:max_len]

    # Добавляем паддинг если нужно
    if len(indices) < max_len:
        indices += [pad_idx] * (max_len - len(indices))
        chars += ['<PAD>'] * (max_len - len(chars))

    return indices, chars

print("✅ Функция токенизации определена")
print()

# ============================================================================
# ЯЧЕЙКА 7: ФУНКЦИЯ СОЗДАНИЯ СКОЛЬЗЯЩЕГО ОКНА (чистая функция)
# ============================================================================
def create_sliding_windows(text, window_size=None, stride=None):
    """
    Создание скользящих окон для длинных текстов

    Параметры:
    - text: предобработанный текст
    - window_size: размер окна (по умолчанию из config)
    - stride: шаг окна (по умолчанию из config)

    Возвращает:
    - windows: список окон (подстрок)
    - starts: начальные позиции окон
    """
    if window_size is None:
        window_size = config.MAX_LEN
    if stride is None:
        stride = config.STRIDE

    windows = []
    starts = []

    for start in range(0, len(text), stride):
        window = text[start:start + window_size]
        if window:  # Добавляем только непустые окна
            windows.append(window)
            starts.append(start)

    print(f"📊 Создано {len(windows)} окон (размер: {window_size}, шаг: {stride})")
    return windows, starts

print("✅ Функция создания скользящего окна определена")
print()

# ============================================================================
# ЯЧЕЙКА 8: ФУНКЦИЯ ПРЕДСКАЗАНИЯ (чистая функция)
# ============================================================================
def predict_entities(model, text, char2idx, idx2label, device):
    """
    Получение предсказаний сущностей для текста

    Параметры:
    - model: обученная модель
    - text: предобработанный текст
    - char2idx: словарь символ->индекс
    - idx2label: словарь индекс->метка
    - device: устройство (CPU/GPU)

    Возвращает:
    - pred_labels: список предсказанных меток для каждого символа
    """
    # Создаем скользящие окна
    windows, starts = create_sliding_windows(text)

    # Инициализируем массив предсказаний
    pred_labels = ['O'] * len(text)

    print(f"🔍 Обработка текста длиной {len(text)} символов...")

    # Обрабатываем каждое окно
    for i, (window, start) in enumerate(zip(windows, starts)):
        # Преобразуем окно в индексы
        indices, _ = text_to_indices(window, char2idx)

        # Создаем тензор
        x_tensor = torch.tensor([indices], dtype=torch.long).to(device)

        # Получаем предсказания
        with torch.no_grad():
            logits = model(x_tensor)
            predictions = torch.argmax(logits, dim=-1)[0].cpu().numpy()

        # Преобразуем индексы в метки
        window_labels = [idx2label[int(pred)] for pred in predictions[:len(window)]]

        # Обновляем общие предсказания
        for j, label in enumerate(window_labels):
            pos = start + j
            if pos < len(pred_labels):
                if pred_labels[pos] == 'O' and label != 'O':
                    pred_labels[pos] = label

    return pred_labels

print("✅ Функция предсказания определена")
print()

# ============================================================================
# ЯЧЕЙКА 9: ФУНКЦИЯ ВИЗУАЛИЗАЦИИ РЕЗУЛЬТАТОВ (чистая функция)
# ============================================================================
def visualize_results_html(text, pred_labels):
    """Визуализация результатов с цветовым кодированием в HTML"""
    if not text or not pred_labels:
        return "<p>Нет данных для визуализации</p>"

    html_parts = []
    current_word = []
    current_labels = []

    for char, label in zip(text, pred_labels):
        if char == ' ':
            if current_word:
                word_text = ''.join(current_word)
                word_label = next((l for l in current_labels if l != 'O'), 'O')
                color = config.LABEL_COLORS.get(word_label, '#ffffff')

                if word_label == 'O':
                    style = f"color:#000000;"
                else:
                    style = f"background-color:{color};color:#ffffff;padding:2px 4px;border-radius:4px;font-weight:bold;"
                    word_text = f"[{word_label[1:]}] {word_text}"

                html_parts.append(f'<span style="{style}">{word_text}</span> ')
                current_word = []
                current_labels = []
            else:
                html_parts.append(' ')
        else:
            current_word.append(char)
            current_labels.append(label)

    if current_word:
        word_text = ''.join(current_word)
        word_label = next((l for l in current_labels if l != 'O'), 'O')
        color = config.LABEL_COLORS.get(word_label, '#ffffff')

        if word_label == 'O':
            style = f"color:#000000;"
        else:
            style = f"background-color:{color};color:#ffffff;padding:2px 4px;border-radius:4px;font-weight:bold;"
            word_text = f"[{word_label[1:]}] {word_text}"

        html_parts.append(f'<span style="{style}">{word_text}</span>')

    result_html = '<div style="font-family: monospace; line-height: 1.8; padding: 20px; background-color: #f5f5f5; border-radius: 10px;">'
    result_html += '<h3 style="color: #333;">Результат распознавания сущностей:</h3>'
    result_html += '<div style="background-color: white; padding: 20px; border-radius: 8px; border: 1px solid #ddd;">'
    result_html += ''.join(html_parts)
    result_html += '</div>'

    result_html += '<div style="margin-top: 20px; padding: 15px; background-color: white; border-radius: 8px; border: 1px solid #ddd;">'
    result_html += '<h4 style="color: #333; margin-bottom: 10px;">Легенда:</h4>'
    result_html += '<div style="display: flex; flex-wrap: wrap; gap: 10px;">'

    for label, color in config.LABEL_COLORS.items():
        if label != 'O':
            desc = config.CLASS_DESCRIPTIONS.get(label, label)
            result_html += f'<div style="display: inline-block; padding: 5px 10px; background-color: {color}; color: white; border-radius: 4px; font-weight: bold;">{label}: {desc}</div>'

    result_html += '</div></div></div>'

    return result_html

def visualize_results_console(text, pred_labels):
    """Визуализация результатов в консоли"""
    print("\n" + "="*80)
    print("РЕЗУЛЬТАТ РАСПОЗНАВАНИЯ СУЩНОСТЕЙ")
    print("="*80)

    entities = defaultdict(list)
    current_entity = None
    current_text = []
    start_pos = 0

    for i, (char, label) in enumerate(zip(text, pred_labels)):
        if label != 'O':
            if label != current_entity:
                if current_entity is not None and current_text:
                    entities[current_entity].append({
                        'text': ''.join(current_text),
                        'start': start_pos,
                        'end': i
                    })
                current_entity = label
                current_text = [char]
                start_pos = i
            else:
                current_text.append(char)
        else:
            if current_entity is not None and current_text:
                entities[current_entity].append({
                    'text': ''.join(current_text),
                    'start': start_pos,
                    'end': i
                })
                current_entity = None
                current_text = []

    if current_entity is not None and current_text:
        entities[current_entity].append({
            'text': ''.join(current_text),
            'start': start_pos,
            'end': len(text)
        })

    print(f"\n📊 Статистика распознавания:")
    print(f"   Всего символов: {len(text)}")

    total_entities = 0
    for label, entity_list in entities.items():
        count = len(entity_list)
        total_entities += count
        desc = config.CLASS_DESCRIPTIONS.get(label, label)
        print(f"   {label} ({desc}): {count} сущностей")

    print(f"   Всего сущностей: {total_entities}")

    if total_entities > 0:
        print(f"\n🔍 Примеры найденных сущностей:")
        for label, entity_list in entities.items():
            if entity_list:
                desc = config.CLASS_DESCRIPTIONS.get(label, label)
                print(f"\n   {label} ({desc}):")
                for i, entity in enumerate(entity_list[:3]):
                    print(f"     {i+1}. '{entity['text']}' (позиции {entity['start']}-{entity['end']})")
                if len(entity_list) > 3:
                    print(f"     ... и еще {len(entity_list) - 3} сущностей")
    else:
        print("\n⚠️ Сущности не обнаружены")

    print(f"\n🎨 Цветовая кодировка:")
    for label, color in config.LABEL_COLORS.items():
        if label != 'O':
            desc = config.CLASS_DESCRIPTIONS.get(label, label)
            print(f"   {color} {label}: {desc}")

print("✅ Функции визуализации определены")
print()

# ============================================================================
# ЯЧЕЙКА 10: ФУНКЦИЯ СОХРАНЕНИЯ РЕЗУЛЬТАТОВ (исправлена ошибка с datetime)
# ============================================================================
def save_results(text, pred_labels, output_path="ner_results.txt"):
    """Сохранение результатов в файл"""
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("РЕЗУЛЬТАТЫ РАСПОЗНАВАНИЯ СУЩНОСТЕЙ\n")
            f.write("=" * 80 + "\n\n")

            f.write(f"Дата обработки: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Длина текста: {len(text)} символов\n\n")

            entities = defaultdict(list)
            current_entity = None
            current_text = []
            start_pos = 0

            for i, (char, label) in enumerate(zip(text, pred_labels)):
                if label != 'O':
                    if label != current_entity:
                        if current_entity is not None and current_text:
                            entities[current_entity].append({
                                'text': ''.join(current_text),
                                'start': start_pos,
                                'end': i
                            })
                        current_entity = label
                        current_text = [char]
                        start_pos = i
                    else:
                        current_text.append(char)
                else:
                    if current_entity is not None and current_text:
                        entities[current_entity].append({
                            'text': ''.join(current_text),
                            'start': start_pos,
                            'end': i
                        })
                        current_entity = None
                        current_text = []

            if current_entity is not None and current_text:
                entities[current_entity].append({
                    'text': ''.join(current_text),
                    'start': start_pos,
                    'end': len(text)
                })

            f.write("СТАТИСТИКА:\n")
            f.write("-" * 40 + "\n")
            total_entities = 0
            for label, entity_list in entities.items():
                count = len(entity_list)
                total_entities += count
                desc = config.CLASS_DESCRIPTIONS.get(label, label)
                f.write(f"{label}: {desc} - {count} сущностей\n")
            f.write(f"Всего сущностей: {total_entities}\n\n")

            if total_entities > 0:
                f.write("НАЙДЕННЫЕ СУЩНОСТИ:\n")
                f.write("-" * 40 + "\n")
                for label, entity_list in entities.items():
                    if entity_list:
                        desc = config.CLASS_DESCRIPTIONS.get(label, label)
                        f.write(f"\n{label} ({desc}):\n")
                        for i, entity in enumerate(entity_list, 1):
                            f.write(f"  {i:3d}. '{entity['text']}' (позиции {entity['start']}-{entity['end']})\n")

            f.write("\n" + "=" * 80 + "\n")
            f.write("ТЕКСТ С ЦВЕТОВОЙ РАЗМЕТКОЙ (HTML)\n")
            f.write("=" * 80 + "\n\n")
            f.write(visualize_results_html(text, pred_labels))

        print(f"✅ Результаты сохранены в файл: {output_path}")
        return output_path

    except Exception as e:
        print(f"❌ Ошибка при сохранении результатов: {e}")
        return None

print("✅ Функция сохранения результатов определена")
print()

# ============================================================================
# ЯЧЕЙКА 11: ФУНКЦИЯ ЗАГРУЗКИ ФАЙЛА (с поддержкой Google Colab)
# ============================================================================
def load_text_from_file(file_path=None):
    """
    Загрузка текста из файла

    Параметры:
    - file_path: путь к файлу

    Возвращает:
    - Текст из файла или None при ошибке
    """
    if file_path is None:
        # Пробуем загрузить пример текста
        sample_path = config.DEFAULT_PATHS["data/sample_contract.txt"]
        if os.path.exists(sample_path):
            print(f"📁 Используется пример текста из файла: {sample_path}")
            file_path = sample_path
        else:
            print("⚠️ Пример текста не найден. Введите текст вручную.")
            return None

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        print(f"✅ Файл загружен: {file_path}")
        print(f"📄 Размер текста: {len(text)} символов")
        return text
    except FileNotFoundError:
        print(f"❌ Файл не найден: {file_path}")
        return None
    except Exception as e:
        print(f"❌ Ошибка загрузки файла: {e}")
        return None

print("✅ Функция загрузки файла определена")
print()

# ============================================================================
# ЯЧЕЙКА 12: ФУНКЦИЯ ДЛЯ ЗАГРУЗКИ ФАЙЛОВ В COLAB (опционально)
# ============================================================================
def upload_files_colab():
    """Функция для загрузки файлов в Google Colab"""
    try:
        from google.colab import files
        print("📁 Загрузите необходимые файлы (модель и словари):")

        # Список необходимых файлов
        required_files = [
            "bilstm_ner_best.pt",
            "char_vocab.json",
            "label_vocab.json"
        ]

        print("Необходимые файлы:")
        for file in required_files:
            print(f"  - {file}")

        uploaded = files.upload()

        if uploaded:
            print("✅ Файлы загружены:")
            for filename in uploaded.keys():
                print(f"  - {filename}")
            return True
        else:
            print("⚠️ Файлы не загружены")
            return False

    except ImportError:
        print("ℹ️ Google Colab не обнаружен. Используйте локальные файлы.")
        return False
    except Exception as e:
        print(f"❌ Ошибка загрузки файлов: {e}")
        return False

print("✅ Функция загрузки файлов в Colab определена")
print()

# ============================================================================
# ЯЧЕЙКА 13: ГЛАВНАЯ ФУНКЦИЯ ИНФЕРЕНСА (исправлена)
# ============================================================================
def run_ner_inference(input_text=None, input_file=None,
                      model_path=None, char_vocab_path=None, label_vocab_path=None,
                      save_output=True, visualize=True):
    """
    Главная функция запуска NER инференса

    Параметры:
    - input_text: текст для обработки (строка)
    - input_file: путь к файлу с текстом
    - model_path: путь к модели
    - char_vocab_path: путь к словарю символов
    - label_vocab_path: путь к словарю меток
    - save_output: сохранять ли результаты в файл
    - visualize: показывать ли визуализацию

    Возвращает:
    - results: словарь с результатами или None при ошибке
    """

    print("\n" + "="*80)
    print("🚀 ЗАПУСК NER ИНФЕРЕНСА")
    print("="*80)

    results = None

    try:
        # 1. ПОЛУЧЕНИЕ ВХОДНЫХ ДАННЫХ
        print("\n1. 📥 Получение входных данных...")

        text = None

        if input_text:
            text = input_text
            print("   Используется предоставленный текст")
        elif input_file:
            text = load_text_from_file(input_file)
        else:
            # Пробуем загрузить из файла или запросить
            text = load_text_from_file()
            if text is None:
                print("   Введите текст для анализа (Ctrl+Enter для завершения):")
                text = input()

        if not text or len(text.strip()) == 0:
            print("❌ Текст не предоставлен")
            return None

        print(f"   Длина текста: {len(text)} символов")

        # 2. ЗАГРУЗКА МОДЕЛИ И СЛОВАРЕЙ
        print("\n2. 🤖 Загрузка модели и словарей...")
        model, char2idx, idx2label, device = load_model_and_vocabs(
            model_path=model_path,
            char_vocab_path=char_vocab_path,
            label_vocab_path=label_vocab_path
        )

        # 3. ПРЕДОБРАБОТКА ТЕКСТА
        print("\n3. 🔧 Предобработка текста...")
        processed_text = preprocess_text(text, lower=True)
        print(f"   Текст предобработан: {len(processed_text)} символов")

        # 4. ПРЕДСКАЗАНИЕ СУЩНОСТЕЙ
        print("\n4. 🔮 Распознавание сущностей...")
        pred_labels = predict_entities(model, processed_text, char2idx, idx2label, device)

        # 5. ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
        if visualize:
            print("\n5. 🎨 Визуализация результатов...")
            visualize_results_console(processed_text, pred_labels)
            html_result = visualize_results_html(processed_text, pred_labels)
            display(HTML(html_result))

        # 6. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ
        output_file = None
        if save_output:
            print("\n6. 💾 Сохранение результатов...")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"ner_results_{timestamp}.txt"
            save_results(processed_text, pred_labels, output_file)

        # 7. ВОЗВРАТ РЕЗУЛЬТАТОВ
        results = {
            'text': text,
            'processed_text': processed_text,
            'predictions': pred_labels,
            'entities_count': sum(1 for label in pred_labels if label != 'O'),
            'output_file': output_file
        }

        print("\n" + "="*80)
        print("✅ ИНФЕРЕНС УСПЕШНО ЗАВЕРШЕН")
        print("="*80)

        return results

    except FileNotFoundError as e:
        print(f"\n❌ КРИТИЧЕСКАЯ ОШИБКА: {e}")
        print("\n💡 РЕШЕНИЕ ПРОБЛЕМЫ:")
        print("1. Убедитесь, что следующие файлы находятся в той же папке, что и ноутбук:")
        for key, filename in config.DEFAULT_PATHS.items():
            if key != "sample_text":
                print(f"   - {filename}")
        print("\n2. Или укажите правильные пути при вызове функции:")
        print('   run_ner_inference(input_text="Ваш текст", model_path="путь/к/модели.pt", ...)')
        return None
    except Exception as e:
        print(f"\n❌ ОШИБКА ВЫПОЛНЕНИЯ: {e}")
        return None

print("✅ Главная функция инференса определена")
print()

# ============================================================================
# ЯЧЕЙКА 14: ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ============================================================================
def check_required_files():
    """Проверка наличия необходимых файлов"""
    print("🔍 Проверка наличия необходимых файлов...")

    missing_files = []

    for key, filename in config.DEFAULT_PATHS.items():
        if key != "sample_text":  # sample_text опционален
            if os.path.exists(filename):
                print(f"   ✅ {filename} - найден")
            else:
                print(f"   ❌ {filename} - ОТСУТСТВУЕТ")
                missing_files.append(filename)

    if missing_files:
        print(f"\n⚠️ Отсутствуют {len(missing_files)} файлов:")
        for file in missing_files:
            print(f"   - {file}")
        print("\n💡 Решения:")
        print("1. Скопируйте файлы в ту же папку, где находится ноутбук")
        print("2. Используйте функцию upload_files_colab() для загрузки в Google Colab")
        print("3. Укажите правильные пути при вызове run_ner_inference()")
        return False
    else:
        print("\n✅ Все необходимые файлы присутствуют!")
        return True

def run_demo_example():
    """Запуск демонстрационного примера"""
    print("\n" + "="*80)
    print("🧪 ДЕМОНСТРАЦИОННЫЙ ПРИМЕР")
    print("="*80)

    demo_text = """Договор №123 от 10 января 2024 года.
Сумма договора: 150000 рублей.
Аванс 20% (30000 рублей) оплачивается в течение 5 банковских дней.
Оставшаяся сумма 120000 рублей оплачивается до 30 марта 2024 года.
Адрес поставки: г. Москва, ул. Ленина, д. 15."""

    print("Текст для анализа:")
    print("-" * 40)
    print(demo_text)
    print("-" * 40)

    results = run_ner_inference(
        input_text=demo_text,
        save_output=False,
        visualize=True
    )

    return results

print("✅ Вспомогательные функции определены")
print()

# ============================================================================
# ЯЧЕЙКА 15: ТЕСТОВАЯ ЯЧЕЙКА ДЛЯ ПОЛЬЗОВАТЕЛЯ (ФИНАЛЬНАЯ)
# ============================================================================
print("="*80)
print("🎯 СИСТЕМА NER ДЛЯ ЮРИДИЧЕСКИХ ДОКУМЕНТОВ - ГОТОВА К ИСПОЛЬЗОВАНИЮ")
print("="*80)
print()

# Проверяем наличие файлов
files_ok = check_required_files()

if files_ok:
    print("\n" + "="*80)
    print("🚀 СИСТЕМА ГОТОВА К РАБОТЕ!")
    print("="*80)

    # Запускаем демо
    print("\nХотите запустить демонстрационный пример? (y/n): ", end="")
    choice = input().strip().lower()

    if choice == 'y':
        run_demo_example()
else:
    print("\n" + "="*80)
    print("⚠️  НЕОБХОДИМО НАСТРОИТЬ СИСТЕМУ")
    print("="*80)

    # Проверяем, находимся ли в Google Colab
    try:
        import google.colab
        is_colab = True
    except:
        is_colab = False

    if is_colab:
        print("\nОбнаружен Google Colab")
        print("Хотите загрузить файлы? (y/n): ", end="")
        choice = input().strip().lower()

        if choice == 'y':
            upload_files_colab()
            # Повторная проверка
            if check_required_files():
                run_demo_example()

print("\n" + "="*80)
print("📝 ИНСТРУКЦИЯ ПО ИСПОЛЬЗОВАНИЮ")
print("="*80)
print()
print("1. БЫСТРЫЙ СТАРТ (если все файлы на месте):")
print('   results = run_ner_inference(input_text="Ваш текст договора")')
print()
print("2. С ЗАГРУЗКОЙ ИЗ ФАЙЛА:")
print("   results = run_ner_inference(input_file='договор.txt')")
print()
print("3. С УКАЗАНИЕМ ПУТЕЙ К ФАЙЛАМ:")
print('   results = run_ner_inference(')
print('       input_text="Текст",')
print('       model_path="модель.pt",')
print('       char_vocab_path="char_vocab.json",')
print('       label_vocab_path="label_vocab.json"')
print('   )')
print()
print("4. БЕЗ ВИЗУАЛИЗАЦИИ (только результаты):")
print('   results = run_ner_inference(input_text="Текст", visualize=False)')
print()
print("5. ПРОВЕРКА ФАЙЛОВ:")
print("   check_required_files()")
print()
print("6. ДЕМО-ПРИМЕР:")
print("   run_demo_example()")
print()
print("📌 Результаты сохраняются в файл ner_results_<дата_время>.txt")

СИСТЕМА NER ДЛЯ ЮРИДИЧЕСКИХ ДОКУМЕНТОВ
✅ Библиотеки загружены

✅ Конфигурация загружена

✅ Архитектура модели определена

✅ Функция загрузки модели определена

✅ Функция предобработки текста определена

✅ Функция токенизации определена

✅ Функция создания скользящего окна определена

✅ Функция предсказания определена

✅ Функции визуализации определены

✅ Функция сохранения результатов определена

✅ Функция загрузки файла определена

✅ Функция загрузки файлов в Colab определена

✅ Главная функция инференса определена

✅ Вспомогательные функции определены

🎯 СИСТЕМА NER ДЛЯ ЮРИДИЧЕСКИХ ДОКУМЕНТОВ - ГОТОВА К ИСПОЛЬЗОВАНИЮ

🔍 Проверка наличия необходимых файлов...
   ❌ bilstm_ner_best.pt - ОТСУТСТВУЕТ
   ❌ char_vocab.json - ОТСУТСТВУЕТ
   ❌ label_vocab.json - ОТСУТСТВУЕТ

⚠️ Отсутствуют 3 файлов:
   - bilstm_ner_best.pt
   - char_vocab.json
   - label_vocab.json

💡 Решения:
1. Скопируйте файлы в ту же папку, где находится ноутбук
2. Используйте функцию upload_files_colab() для загрузки в G

Saving bilstm_ner_best (1).pt to bilstm_ner_best (1).pt
✅ Файлы загружены:
  - bilstm_ner_best (1).pt
🔍 Проверка наличия необходимых файлов...
   ❌ bilstm_ner_best.pt - ОТСУТСТВУЕТ
   ❌ char_vocab.json - ОТСУТСТВУЕТ
   ❌ label_vocab.json - ОТСУТСТВУЕТ

⚠️ Отсутствуют 3 файлов:
   - bilstm_ner_best.pt
   - char_vocab.json
   - label_vocab.json

💡 Решения:
1. Скопируйте файлы в ту же папку, где находится ноутбук
2. Используйте функцию upload_files_colab() для загрузки в Google Colab
3. Укажите правильные пути при вызове run_ner_inference()

📝 ИНСТРУКЦИЯ ПО ИСПОЛЬЗОВАНИЮ

1. БЫСТРЫЙ СТАРТ (если все файлы на месте):
   results = run_ner_inference(input_text="Ваш текст договора")

2. С ЗАГРУЗКОЙ ИЗ ФАЙЛА:
   results = run_ner_inference(input_file='договор.txt')

3. С УКАЗАНИЕМ ПУТЕЙ К ФАЙЛАМ:
   results = run_ner_inference(
       input_text="Текст",
       model_path="модель.pt",
       char_vocab_path="char_vocab.json",
       label_vocab_path="label_vocab.json"
   )

4. БЕЗ ВИЗУАЛИЗАЦИИ 